In [92]:
import pandas as pd
import numpy as np
import difflib
import os

In [93]:
cd C:\Users\Diego\Documents\DTU\Spring2024\MBML\european_vote_analysis

C:\Users\Diego\Documents\DTU\Spring2024\MBML\european_vote_analysis


In [94]:
data_european = pd.read_csv('data/raw/COMEPELDA_aggregate_v1.00.csv')
data_national = pd.read_csv('data/raw/1999-2019_CHES_dataset_means.csv')
country_codes = pd.read_csv('data/raw/countries.csv')  
party_names = pd.read_csv('data/raw/partyNames_ncv2.csv')  

In [95]:
# Add country codes to national data
data_european = pd.merge(data_european, country_codes, left_on='cName', right_on='Country', how='left')
# drop 2006, 2007, 2013 years due to particular countries elections
data_european = data_european.drop(data_european[data_european['ElYear'].isin([2006, 2007, 2013])].index)

In [96]:
# add country codes to european data
data_national = pd.merge(data_national, country_codes, left_on='country', right_on='ID', how='left')

In [97]:
data_national = pd.merge(data_national, party_names, left_on='party_id', right_on='partyID', how='left')

In [ ]:
test_data = pd.merge(data_european, data_national, left_on=["Country", 'partyName'], right_on=["cName", 'pName'], how='outer', indicator=True)

In [97]:
T1 = test_data[test_data['_merge'] == "left_only"]
T2 = test_data[test_data['_merge'] == "right_only"]
T3 = test_data[test_data['_merge'] == "both"]

In [ ]:
data_european["alt_partyName"] = data_european[data_european["Country"] == "Belgium"]["partyName"].apply(lambda x: difflib.get_close_matches(x, data_national[data_national["cName"] == "Belgium"]["pName"]))

In [122]:
data_european["alt_partyName"] = data_european["partyName"].astype(str).apply(lambda x: difflib.get_close_matches(x, data_national["pName"].astype(str)))

In [18]:
score = difflib.SequenceMatcher(None, x, result).ratio()
score

0.0

In [49]:
data_national[data_national.partyName == "Partito dei Pensionati"]

,country,eastwest,eumember,year,expert,party_id,cmp_id,party,vote,seat,...,mip_three,chesversion,ID,Abbreviation,Country,CountryAbrev,partyID,partyAbrev,partyName,partyNameEng
546,8,1,1,2006,6,830,NaN,PP,0.9,0.0,...,NaN,2020.1,8,IT,Italy,IT,830.0,PP,Partito dei Pensionati,Pensioners’ Party


In [89]:
a.to_csv("data/processed/MISSINGpARTIES.csv", index=False)

In [101]:
year_corr = {1999: 1999, 
 2004: 2002,
 2009: 2006,
 2014: 2014}
eu_unique_columns = ["Abbreviation", "pName"]
data_unique_columns = ["Abbreviation", "partyName"]
parties_data_eu = data_european[eu_unique_columns].drop_duplicates()
parties_data_nat = data_national[data_unique_columns].drop_duplicates()
for row in parties_data_eu.itertuples():
    y = parties_data_nat[(parties_data_nat["Abbreviation"] == row.Abbreviation)]["partyName"].astype(str).str.strip()
    x = row.pName
    result = difflib.get_close_matches(x, y, n=1, cutoff=0.6)
    if result:
        parties_data_eu.loc[row.Index, "correspondance"] = ", ".join(result)
        parties_data_eu.loc[row.Index, "score"] = difflib.SequenceMatcher(None, x, result[0]).ratio()

    else: 
        parties_data_eu.loc[row.Index, "correspondance"] = np.nan

In [104]:
parties_data_eu.to_csv("data/processed/parties_correspondance.csv", index=False)

In [113]:
data_european[data_european["IndepCand01"] == 1, ]

,IDAD,cName,ElYear,IDpty,pName,dName,dNameEng,cValid,dValid,pcAbs,pcShare,pdAbs,pdShare,pdNofCandSel,pdNofCandEl,pdFlagNofCand,cSeats,dMag,pcSeats,pdSeats,pDistricted01,cFormula,cBetwPtyThresh,cAppar01,cMultiTier01,cFormulaBetwDist,cBallotType,cPanach01,cRankedList01,cPrefVotOblig01,cPrefVotN,cPrefThrFormal,pdPrefThrEmp,pdPrefVotSum,pdPrefVotLoser,pdListFlex,pdPat,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxAlliance,pApparParty1,pApparParty2,IDauxElec,IDauxIndep,ID,Abbreviation,Country
184,185,Estonia,2009,-8897136,Indep. cand.: Indrek Tarand,(nationwide),(nationwide),396982,396982,102460,0.25809734446398075,102460,0.25809734446398075,1,1,0,6,6,1,1,0,d'Hondt,(none),0,0,.,closed,.,1,.,.,.,.,.,.,.,LR,1,0,0,0,.,.,.,395,2029,22,EST,Estonia
190,191,Estonia,2014,-8897230,Indep. cand.: Kristiina Ojuland,(nationwide),(nationwide),328493,328493,3024,0.009205675615614336,3024,0.009205675615614336,1,1,0,6,6,0,0,0,d'Hondt,(none),0,0,.,open,0,1,1,1,.,.,3024,.,.,PV,1,0,0,0,.,.,.,822,.,22,EST,Estonia
191,192,Estonia,2014,-8897136,Indep. cand.: Indrek Tarand,(nationwide),(nationwide),328493,328493,43369,0.1320241222796224,43369,0.1320241222796224,1,1,0,6,6,1,1,0,d'Hondt,(none),0,0,.,open,0,1,1,1,.,.,43369,.,.,PV,1,0,0,0,.,.,.,822,2029,22,EST,Estonia
531,532,Great Britain,1999,-881178,Indep. cand.: Christine Oddy,West Midlands (UK),West Midlands (UK),10002270,848684,36849,0.0036840637175361195,36849,0.0434189875147876,1,1,0,84,8,0,0,1,d'Hondt,(none),0,0,.,closed,.,1,.,.,.,.,.,.,.,LR,1,0,0,0,.,.,.,686,.,11,UK,Great Britain
845,846,Ireland,1999,-884389,Indep. cand.: Dana Rosemary Scallon,Connacht-Ulster,Connacht-Ulster,1391740,320151,51086,0.036706568755658385,51086,0.15956845363594055,1,1,0,15,3,1,1,1,STV-Droop,(none),0,0,.,STV,.,0,1,1,.,.,51086,.,.,.,1,0,0,0,.,.,.,491,30,7,IRL,Ireland
846,847,Ireland,1999,-881287,Indep. cand.: Pat Cox,Munster,Munster,1391740,448579,63954,0.04595254860821705,63954,0.1425702050252018,1,1,0,15,4,1,1,1,STV-Droop,(none),0,0,.,STV,.,0,1,1,.,.,63954,.,.,.,1,0,0,0,.,.,.,491,2328,7,IRL,Ireland
862,863,Ireland,2004,-8828119,Indep. cand.: Kathy Sinnott,South (IE),South (IE),1780768,484270,89127,0.05004975381408471,89127,0.18404402502736078,1,1,0,13,3,1,1,1,STV-Droop,(none),0,0,.,STV,.,0,1,1,.,.,89127,.,.,.,1,0,0,0,.,.,.,123,30,7,IRL,Ireland
863,864,Ireland,2004,-8828116,Indep. cand.: Marian Harkin,North-West (IE),North-West (IE),1780768,421423,66664,0.03743553343276609,66664,0.1581878540089174,1,1,0,13,3,1,1,1,STV-Droop,(none),0,0,.,STV,.,0,1,1,.,.,66664,.,.,.,1,0,0,0,.,.,.,123,2329,7,IRL,Ireland
864,865,Ireland,2004,-884389,Indep. cand.: Dana Rosemary Scallon,North-West (IE),North-West (IE),1780768,421423,56992,0.032004168987762587,56992,0.135237042116828,1,1,0,13,3,0,0,1,STV-Droop,(none),0,0,.,STV,.,0,1,1,.,.,56992,.,.,.,1,0,0,0,.,.,.,123,.,7,IRL,Ireland
884,885,Ireland,2009,-8828119,Indep. cand.: Kathy Sinnott,South (IE),South (IE),1829313,498127,58485,0.031971018628304725,58485,0.11740981717513807,1,1,0,12,3,0,0,1,STV-Droop,(none),0,0,.,STV,.,0,1,1,.,.,58485,.,.,.,1,0,0,0,.,.,.,369,.,7,IRL,Ireland


In [111]:
data_european.drop(columns=["Country", "cName", "ID"])

,IDAD,ElYear,IDpty,pName,dName,dNameEng,cValid,dValid,pcAbs,pcShare,pdAbs,pdShare,pdNofCandSel,pdNofCandEl,pdFlagNofCand,cSeats,dMag,pcSeats,pdSeats,pDistricted01,cFormula,cBetwPtyThresh,cAppar01,cMultiTier01,cFormulaBetwDist,cBallotType,cPanach01,cRankedList01,cPrefVotOblig01,cPrefVotN,cPrefThrFormal,pdPrefThrEmp,pdPrefVotSum,pdPrefVotLoser,pdListFlex,pdPat,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxAlliance,pApparParty1,pApparParty2,IDauxElec,IDauxIndep,Abbreviation
0,1,1999,50,Freiheitliche Partei Österreichs,(nationwide),(nationwide),2801353,2801353,655519,0.23400085601493278,655519,0.23400085601493278,42,42,0,21,21,5,5,0,d'Hondt,4%,0,0,.,flexible,0,1,0,1,7%,45886.33,29408,427,weakly,LR,0,0,0,0,.,.,.,16,.,AUS
1,2,1999,955,Liberales Forum,(nationwide),(nationwide),2801353,2801353,74467,0.026582512093263504,74467,0.026582512093263504,40,40,0,21,21,0,0,0,d'Hondt,4%,0,0,.,flexible,0,1,0,1,7%,.,9212,6939,.,.,0,0,0,0,.,.,.,16,.,AUS
2,3,1999,973,Sozialdemokratische Partei Österreichs,(nationwide),(nationwide),2801353,2801353,888338,0.31711033918253073,888338,0.31711033918253073,42,42,0,21,21,7,7,0,d'Hondt,4%,0,0,.,flexible,0,1,0,1,7%,62183.66,46525,384,weakly,LR,0,0,0,0,.,.,.,16,.,AUS
3,4,1999,1013,Österreichische Volkspartei,(nationwide),(nationwide),2801353,2801353,859175,0.3067000124582657,859175,0.3067000124582657,42,42,0,21,21,7,7,0,d'Hondt,4%,0,0,.,flexible,0,1,0,1,7%,60142.25000000001,80782,1633,weakly,LR,0,0,0,0,.,.,.,16,.,AUS
4,5,1999,1429,Die Grünen – Die Grüne Alternative,(nationwide),(nationwide),2801353,2801353,260273,0.0929097475398495,260273,0.0929097475398495,42,42,0,21,21,2,2,0,d'Hondt,4%,0,0,.,flexible,0,1,0,1,7%,18219.11,48137,514,weakly,LR,0,0,0,0,.,.,.,16,.,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,1802,2014,904,Socialdemokraterna,(nationwide),(nationwide),3716778,3716778,899074,0.2418960723508372,899074,0.2418960723508372,30,30,0,20,20,5,5,0,Modified Sainte-Laguë,4%,0,0,.,flexible,0,1,0,1,5%,44953.69921875,438882,32283,weakly,LR,0,0,0,0,.,.,.,843,.,SV
1802,1803,2014,1154,Miljöpartiet de Gröna,(nationwide),(nationwide),3716778,3716778,572591,0.15405574398040453,572591,0.15405574398040453,40,40,0,20,20,4,4,0,Modified Sainte-Laguë,4%,0,0,.,flexible,0,1,0,1,5%,28629.55078125,286744,15316,strongly,MX,0,0,0,0,.,.,.,843,.,SV
1803,1804,2014,1461,Centerpartiet,(nationwide),(nationwide),3716778,3716778,241101,0.0648682810757059,241101,0.0648682810757059,43,43,0,20,20,1,1,0,Modified Sainte-Laguë,4%,0,0,.,flexible,0,1,0,1,5%,12055.0498046875,121605,28452,strongly,PV,0,0,0,0,.,.,.,843,.,SV
1804,1805,2014,1521,Feministiskt initiativ,(nationwide),(nationwide),3716778,3716778,204005,0.05488759350168345,204005,0.05488759350168345,9,9,0,20,20,1,1,0,Modified Sainte-Laguë,4%,0,0,.,flexible,0,1,0,1,5%,10200.25,112051,7301,weakly,LR,0,0,0,0,.,.,.,843,.,SV


In [66]:
import matplotlib.pyplot as plt
a = parties_data_eu[parties_data_eu["score"] <= 0.7]

In [62]:
x = "Partito Pensionati"
y = ["Partito dei Pensionati", "App"]

result = difflib.get_close_matches(x, y)
difflib.SequenceMatcher(None, x, result[0]).ratio()

0.9

In [103]:
parties_data_eu.to_csv("data/processed/party_correspondance.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'data/processed/party_correspondance.csv'

In [68]:
data_european["cValid"]

0       2801353
1       2801353
2       2801353
3       2801353
4       2801353
         ...   
1801    3716778
1802    3716778
1803    3716778
1804    3716778
1805    3716778
Name: cValid, Length: 1774, dtype: object

In [82]:
data_european.loc[(data_european["Abbreviation"] == "ESP") &
                (data_european["ElYear"] == 2014), 
                ["pName", "pcShare"]]

,pName,pcShare
1761,Partido Communista | Izquierda Unida,0.1002728415700968
1762,Bloque Nacionalista Galego,0.020780363554517647
1763,Unión Progreso y Democracia,0.06506797869615542
1764,Alianza-Partido Popular,0.260870951742484
1765,Esquerra Republicana de Catalunya,0.0401058776021921
1766,Coalición por la Europa de los Pueblos,0.054230381046320435
1767,Partido Socialista Obrero Español,0.23005616218134747
1768,Chunta Aragonesista,0.019240091924897788
1769,Ciudadanos – Partido de la Ciudadanía,0.031644759053599265
1770,Podemos,0.07981029668847328


In [110]:
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 100)


In [85]:
data_national.loc[(data_national["Abbreviation"] == "ESP") &
                (data_national["year"] == 2014), 
                ["partyName", "vote"]]

,partyName,vote
331,Eusko Alkartasuna,0.2
332,Euzko Alderdi Jeltzalea/Partido Nacionalista Vasco,1.4
333,Convergència i Unió,4.2
334,Unión Progreso y Democracia,4.8
335,Partido Popular,41.9
336,Iniciativa per Catalunya,1.3
337,Ciudadanos—Partido de la Ciudadanía,NaN
338,Partido Socialista Obrero Español,28.8
339,Coalición Canaria,0.6
340,Bloque Nacionalista Galego,0.8


Index(['country', 'eastwest', 'eumember', 'year', 'expert', 'party_id',
       'cmp_id', 'party', 'vote', 'seat', 'electionyear', 'epvote', 'family',
       'govt', 'eu_position', 'eu_salience', 'eu_dissent', 'eu_blur',
       'eu_benefit', 'eu_ep', 'eu_fiscal', 'eu_intmark', 'eu_employ',
       'eu_budgets', 'eu_agri', 'eu_cohesion', 'eu_environ', 'eu_asylum',
       'eu_foreign', 'eu_turkey', 'lrgen', 'lrecon', 'lrecon_salience',
       'lrecon_dissent', 'lrecon_blur', 'galtan', 'galtan_salience',
       'galtan_dissent', 'galtan_blur', 'spendvtax', 'spendvtax_salience',
       'deregulation', 'dereg_salience', 'redistribution', 'redist_salience',
       'econ_interven', 'civlib_laworder', 'civlib_salience',
       'sociallifestyle', 'social_salience', 'religious_principles',
       'relig_salience', 'immigrate_policy', 'immigrate_salience',
       'immigrate_dissent', 'multiculturalism', 'multicult_salience',
       'multicult_dissent', 'urban_rural', 'urban_salience', 'environment'